In [1]:
import llama_index
import ollama
import os

from utils import *

In [3]:

chroma_collection = chroma_init("./chroma_db")

storage_context = create_llama_connectors(chroma_collection)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

llm = Ollama(model="deepseek-r1:latest", request_timeout=1200.0, device='cuda')
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5", device="cuda")

index = VectorStoreIndex.from_vector_store(
    vector_store = vector_store,
    storage_context = storage_context,
    embed_model=embed_model
)

## Chat Engine

In [5]:
chat = index.as_chat_engine(
    llm=llm,
    similarity_top_k=3,
    streaming=True,
    verbose=True
)

In [6]:
chat.chat("Hola")

> Running step afc2b29f-5d3d-475b-b3a9-4fa753aba7e1. Step input: Hola
Thought: The user has greeted me with "Hola," which is Spanish for "Hello." Since my instructions are primarily in English but I can process some Spanish queries, I might need to clarify the language they want their response in. However, as a general approach, responding in English unless specified otherwise makes sense.
Answer: Sure! How can I assist you today?


AgentChatResponse(response='Sure! How can I assist you today?', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [4]:
def wrapper_chat_history(memory):
    chat_history = []
    for m in memory: 
        if m.role in ['user','assistant'] and m.content is not None:
            chat_history.append(m.content)
    return chat_history



In [7]:
wrapper_chat_history(chat.chat_history)

['Hola', 'Sure! How can I assist you today?']

In [12]:
def converse(message, chat_history):
    response = chat.chat(message)
    chat_history = wrapper_chat_history(chat.chat_history)

    return response.response

In [8]:
chat.reset()

In [9]:
chat.chat_history

[]

In [13]:
import gradio as gr 

try:
    demo.close()
except:
    pass

demo = gr.ChatInterface(fn=converse)

demo.launch(share=False)

/home/hm/anaconda3/envs/local-rag/lib/python3.11/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


> Running step f39609de-ff02-416d-a4ee-bb6aab8f7bc0. Step input: hola, soy hans, tu companhero para desarmar bombas, me gusta las naranjas
Thought: (Implicit) I can answer without any more tools!
Answer: <think>
Okay, so I'm trying to figure out what Hans wants. He greeted me in Spanish with "hola, soy hans, tucompanhero para desarmar bombas, me gusta las naranjas." Translating that, it means he's my partner to defuse bombs and likes oranges.

Hmm, he mentioned being a partner for bomb disposal. That makes me think he might be involved in some kind of security or protective work. Maybe he needs assistance with something related to that? He also said he likes oranges, so perhaps that's just a casual statement about his preferences.

I'm not sure if I can provide help directly since my tools are set up for certain tasks like answering questions or providing summaries. But maybe there's a way to infer what he needs without direct communication. If someone is involved in bomb disposal, the